# 📚 KM 文件管理系統教學手冊

本教學將以**勞動基準法**為實際案例，帶您學習如何使用 Python 操作 KM 文件管理系統，包含 MongoDB 連線、文件上傳、版本控管、查詢與下載等功能。

## 目錄
1. [環境準備](#1-環境準備)
2. [MongoDB 連線](#2-mongodb-連線)
3. [匯入勞基法 JSON](#3-匯入勞基法-json)
4. [搜尋文件](#4-搜尋文件)
5. [查看 Metadata](#5-查看-metadata)
6. [版本控管](#6-版本控管)
7. [下載文件](#7-下載文件)
8. [歸檔與恢復](#8-歸檔與恢復)
9. [統計與報表](#9-統計與報表)
10. [進階應用](#10-進階應用)

---
## 1. 環境準備

### 1.1 安裝必要套件

In [ ]:
# 安裝 pymongo（如尚未安裝）
!pip install pymongo

### 1.2 設定專案路徑

In [ ]:
import sys
import os
import json

# 取得專案根目錄
project_root = os.path.dirname(os.getcwd()) if 'notebooks' in os.getcwd() else os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"📂 專案根目錄: {project_root}")

### 1.3 確認 MongoDB 狀態

請確認 MongoDB 服務已啟動：

```bash
# macOS
brew services start mongodb/brew/mongodb-community@7.0

# Windows: MongoDB 服務應自動啟動

# Linux
sudo systemctl start mongod
```

---
## 2. MongoDB 連線

### 2.1 建立連線

In [ ]:
from db.connection import get_db_connection
from services.document_service import DocumentService

# 建立連線
conn = get_db_connection()

# 測試連線
if conn.ping():
    print("✅ MongoDB 連線成功！")
    print(f"📂 資料庫: {conn.db.name}")
    print(f"📋 現有 Collections: {conn.db.list_collection_names()}")
else:
    print("❌ 連線失敗，請確認 MongoDB 服務已啟動")

In [ ]:
# 建立文件服務
service = DocumentService()
print("✅ DocumentService 已初始化")

---
## 3. 匯入勞基法 JSON

### 3.1 載入勞基法 JSON 資料

In [ ]:
# 載入勞基法 JSON
json_path = os.path.join(project_root, "data", "勞動基準法.json")

with open(json_path, "r", encoding="utf-8") as f:
    labor_law = json.load(f)

print(f"📜 法規名稱: {labor_law['law_name']}")
print(f"📅 版本: {labor_law['version']}")
print(f"📊 收錄條文數: {len(labor_law['articles'])} 條")
print(f"\n🏷️ 關鍵字: {', '.join(labor_law['metadata']['keywords'])}")

### 3.2 查看 JSON 結構

In [ ]:
# 查看第一條條文結構
first_article = labor_law['articles'][0]

print("📄 條文結構範例（第1條）：\n")
print(json.dumps(first_article, ensure_ascii=False, indent=2))

### 3.3 查看特別休假條文（含結構化資料）

In [ ]:
# 找到第38條 - 特別休假
article_38 = next(a for a in labor_law['articles'] if a['article_id'] == '第38條')

print("📋 第38條 特別休假\n")
print(f"章節: {article_38['chapter']}")
print(f"重要性: {article_38['importance']}")
print(f"\n特休天數對照表:")
print("-" * 40)

for leave in article_38['annual_leave']:
    print(f"  {leave['tenure']}: {leave['days']} 日")

---
## 4. 搜尋文件

### 4.1 列出所有已匯入的勞基法條文

In [ ]:
# 取得所有文件
all_docs = service.get_all()

print(f"📚 系統中共有 {len(all_docs)} 份文件：\n")
print(f"{'編號':<15} {'標題':<30} {'版本':>6}")
print("=" * 55)

for doc in all_docs:
    print(f"{doc.doc_code:<15} {doc.title:<30} v{doc.current_version:>5}")

### 4.2 關鍵字搜尋

In [ ]:
# 搜尋「休假」相關條文
results = service.search(keyword="休假")

print(f"🔍 搜尋「休假」找到 {len(results)} 筆結果：\n")
for doc in results:
    print(f"  • {doc.doc_code}: {doc.title}")

In [ ]:
# 搜尋「工資」相關條文
results = service.search(keyword="工資")

print(f"🔍 搜尋「工資」找到 {len(results)} 筆結果：\n")
for doc in results:
    print(f"  • {doc.doc_code}: {doc.title}")

In [ ]:
# 搜尋「退休」相關條文
results = service.search(keyword="退休")

print(f"🔍 搜尋「退休」找到 {len(results)} 筆結果：\n")
for doc in results:
    print(f"  • {doc.doc_code}: {doc.title}")

---
## 5. 查看 Metadata

### 5.1 查看條文詳細資訊

In [ ]:
# 取得第38條（特別休假）詳細資訊
doc = service.get_by_doc_code("LAW-LSA-038")

print("=" * 50)
print(f"📄 {doc.title}")
print("=" * 50)
print(f"\n編號: {doc.doc_code}")
print(f"部門: {doc.department}")
print(f"分類: {doc.category}")
print(f"狀態: {doc.status}")
print(f"建立時間: {doc.created_at}")

In [ ]:
# 查看 Metadata
print("\n📋 Metadata 資訊：\n")

important_keys = ['law_name', 'law_version', 'chapter', 'article_id', 
                  'article_title', 'importance', 'keywords']

for key in important_keys:
    if key in doc.metadata:
        value = doc.metadata[key]
        if isinstance(value, list):
            value = ', '.join(value)
        print(f"  {key}: {value}")

In [ ]:
# 查看特休天數結構化資料
if 'annual_leave' in doc.metadata:
    print("\n📊 特別休假天數對照表：\n")
    print(f"{'年資':<25} {'天數':>10}")
    print("-" * 40)
    
    for leave in doc.metadata['annual_leave']:
        days = leave['days']
        if isinstance(days, int):
            days = f"{days} 日"
        print(f"{leave['tenure']:<25} {days:>10}")

### 5.2 查看基本工資資訊

In [ ]:
# 取得第21條（基本工資）
doc_21 = service.get_by_doc_code("LAW-LSA-021")

print(f"📄 {doc_21.title}\n")

if 'current_minimum_wage' in doc_21.metadata:
    wage = doc_21.metadata['current_minimum_wage']
    print("💰 現行基本工資：\n")
    print(f"   月薪: NT$ {wage['monthly']:,}")
    print(f"   時薪: NT$ {wage['hourly']}")
    print(f"   生效日: {wage['effective_date']}")

### 5.3 查看工時規定

In [ ]:
# 取得第30條（正常工時）
doc_30 = service.get_by_doc_code("LAW-LSA-030")

print(f"📄 {doc_30.title}\n")

if 'working_hours' in doc_30.metadata:
    hours = doc_30.metadata['working_hours']
    print("⏰ 正常工作時間：\n")
    print(f"   每日上限: {hours['daily_max']} 小時")
    print(f"   每週上限: {hours['weekly_max']} 小時")

In [ ]:
# 取得第32條（延長工時/加班）
doc_32 = service.get_by_doc_code("LAW-LSA-032")

print(f"📄 {doc_32.title}\n")

if 'overtime_limits' in doc_32.metadata:
    ot = doc_32.metadata['overtime_limits']
    print("⏱️ 加班時數限制：\n")
    print(f"   每月上限: {ot['monthly_max']} 小時")
    print(f"   必要條件: {', '.join(ot['conditions'])}")

---
## 6. 版本控管

### 6.1 查看版本歷史

In [ ]:
# 取得第1條的版本歷史
versions = service.get_version_history("LAW-LSA-001")

print(f"📜 LAW-LSA-001 版本歷史（共 {len(versions)} 版）：\n")

for v in versions:
    print(f"  v{v.version}")
    print(f"    檔案: {v.file_name}")
    print(f"    大小: {v.file_size} bytes")
    print(f"    上傳者: {v.uploaded_by}")
    print(f"    時間: {v.uploaded_at}")
    print(f"    說明: {v.description}")
    print()

### 6.2 上傳新版本（模擬法規修正）

In [ ]:
# 建立修正版內容
updated_content = """勞動基準法
第一章 總則

第1條（立法目的）

為規定勞動條件最低標準，保障勞工權益，加強勞雇關係，
促進社會與經濟發展，特制定本法；本法未規定者，適用其他法律之規定。
雇主與勞工所訂勞動條件，不得低於本法所定之最低標準。

【113年修正說明】
本次修正新增勞工權益保障相關文字說明。

---
來源：全國法規資料庫
版本：民國113年修正（模擬）
"""

# 寫入暫存檔
temp_file = "/tmp/LAW-LSA-001-v2.txt"
with open(temp_file, "w", encoding="utf-8") as f:
    f.write(updated_content)

print("✅ 已建立修正版檔案")

In [ ]:
# 上傳新版本
doc = service.upload_new_version(
    doc_code="LAW-LSA-001",
    file_path=temp_file,
    uploaded_by="法務組",
    description="113年修正版（模擬）- 新增權益保障說明"
)

print(f"\n📄 目前版本: v{doc.current_version}")

# 清理暫存檔
os.remove(temp_file)

In [ ]:
# 再次查看版本歷史
versions = service.get_version_history("LAW-LSA-001")

print(f"📜 更新後的版本歷史（共 {len(versions)} 版）：\n")

for v in versions:
    print(f"  v{v.version}: {v.description} (by {v.uploaded_by})")

---
## 7. 下載文件

### 7.1 下載最新版本

In [ ]:
# 下載第38條最新版本
file_path = service.download_file(
    doc_code="LAW-LSA-038",
    save_path="downloaded_article_38.txt"
)

# 讀取並顯示內容
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

print(f"📥 已下載: {file_path}\n")
print("📄 檔案內容：")
print("=" * 50)
print(content)
print("=" * 50)

In [ ]:
# 清理下載的檔案
os.remove(file_path)
print("✅ 已清理下載檔案")

### 7.2 下載指定版本

In [ ]:
# 下載第1條的第一版（原始版本）
file_path = service.download_file(
    doc_code="LAW-LSA-001",
    version_num=1,
    save_path="downloaded_article_1_v1.txt"
)

with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

print(f"📥 已下載 v1: {file_path}\n")
print("📄 第一版內容：")
print("=" * 50)
print(content[:500])  # 只顯示前500字
print("=" * 50)

os.remove(file_path)

---
## 8. 歸檔與恢復

### 8.1 歸檔文件

In [ ]:
# 選擇一條較少使用的條文進行歸檔示範
doc = service.archive_document("LAW-LSA-044")  # 童工定義
print(f"\n文件狀態: {doc.status}")

In [ ]:
# 搜尋時預設不會顯示已歸檔文件
active_docs = service.search()
print(f"現有文件（不含歸檔）: {len(active_docs)} 筆")

# 包含歸檔文件
all_docs = service.search(include_archived=True)
print(f"所有文件（含歸檔）: {len(all_docs)} 筆")

### 8.2 恢復歸檔文件

In [ ]:
# 恢復歸檔文件
doc = service.restore_document("LAW-LSA-044")
print(f"\n文件狀態: {doc.status}")

---
## 9. 統計與報表

### 9.1 系統統計

In [ ]:
stats = service.get_statistics()

print("=" * 50)
print("📊 KM 文件管理系統統計報表")
print("=" * 50)
print(f"\n📄 總文件數: {stats['total_documents']}")
print(f"📝 總版本數: {stats['total_versions']}")

print("\n📂 依部門統計:")
for dept, count in stats['by_department'].items():
    print(f"   {dept}: {count} 份")

print("\n📋 依分類統計:")
for cat, count in stats['by_category'].items():
    print(f"   {cat}: {count} 份")

print("\n🏷️ 依狀態統計:")
for status, count in stats['by_status'].items():
    print(f"   {status}: {count} 份")

### 9.2 依章節統計勞基法條文

In [ ]:
# 統計各章節條文數
all_docs = service.get_all()
chapter_stats = {}

for doc in all_docs:
    if 'chapter' in doc.metadata:
        chapter = doc.metadata['chapter']
        chapter_stats[chapter] = chapter_stats.get(chapter, 0) + 1

print("📊 各章節條文統計：\n")
for chapter, count in sorted(chapter_stats.items()):
    print(f"  {chapter}: {count} 條")

---
## 10. 進階應用

### 10.1 依重要性篩選條文

In [ ]:
# 找出所有高重要性的條文
all_docs = service.get_all()
high_importance = []

for doc in all_docs:
    if doc.metadata.get('importance') == 'high':
        high_importance.append(doc)

print(f"⭐ 高重要性條文（共 {len(high_importance)} 條）：\n")
for doc in high_importance:
    print(f"  • {doc.doc_code}: {doc.title}")

### 10.2 建立條文索引表

In [ ]:
# 建立完整索引
print("📖 勞動基準法條文索引\n")
print(f"{'條號':<10} {'標題':<20} {'章節':<25} {'重要性':>8}")
print("=" * 70)

for doc in sorted(all_docs, key=lambda x: x.doc_code):
    if doc.doc_code == 'LAW-LSA-000':  # 跳過總覽
        continue
    
    article_id = doc.metadata.get('article_id', '-')
    title = doc.metadata.get('article_title', doc.title)[:18]
    chapter = doc.metadata.get('chapter', '-')[:23]
    importance = doc.metadata.get('importance', '-')
    
    print(f"{article_id:<10} {title:<20} {chapter:<25} {importance:>8}")

### 10.3 快速查詢特定主題

In [ ]:
def query_by_topic(topic):
    """依主題查詢相關條文"""
    results = service.search(keyword=topic)
    
    print(f"🔍 主題「{topic}」相關條文：\n")
    
    if not results:
        print("  （無相關結果）")
        return
    
    for doc in results:
        print(f"  📄 {doc.title}")
        print(f"     文件編號: {doc.doc_code}")
        if 'keywords' in doc.metadata:
            print(f"     關鍵字: {', '.join(doc.metadata['keywords'])}")
        print()

# 測試查詢
query_by_topic("職災")

In [ ]:
query_by_topic("加班")

---
## 📖 總結

恭喜您完成了 KM 文件管理系統的教學！透過勞動基準法的實際案例，您已經學會：

| 功能 | 對應方法 | 實際應用 |
|------|----------|----------|
| 連線資料庫 | `get_db_connection()` | 連線 MongoDB |
| 搜尋文件 | `service.search()` | 搜尋「休假」「工資」等關鍵字 |
| 查詢詳情 | `service.get_by_doc_code()` | 查看第38條特休天數 |
| 版本歷史 | `service.get_version_history()` | 追蹤法規修正紀錄 |
| 上傳新版 | `service.upload_new_version()` | 上傳法規修正版 |
| 下載文件 | `service.download_file()` | 下載條文檔案 |
| 歸檔/恢復 | `archive_document()` / `restore_document()` | 管理舊版法規 |
| 統計資訊 | `service.get_statistics()` | 系統使用報表 |

### Metadata 結構化資料

勞基法條文的 Metadata 包含豐富的結構化資料：
- `annual_leave`: 特休天數對照表
- `working_hours`: 工時上限
- `overtime_limits`: 加班時數限制
- `current_minimum_wage`: 基本工資
- `voluntary_retirement`: 自請退休條件
- `compensation_types`: 職災補償類型

這些結構化資料可用於未來的 **RAG 智能檢索**擴充。